In [ ]:
import os

os.environ.update({"TF_CPP_MIN_LOG_LEVEL": "3", "CUDA_VISIBLE_DEVICES": ""})

import numpy as np
import tensorflow as tf
from matplotlib.cm import get_cmap
from PIL import Image
from scipy.ndimage import gaussian_filter

import chiron


In [ ]:
results_dir = "../results/combined-4-class"
epoch = 10
tfrecord_file = "../tfrecord/cheng-et-al/fold-1/val.tfrecord"
batch_size = 32


In [ ]:
model = chiron.utils.load_model(os.path.join(results_dir, "model.json"))
model.load_weights(
    os.path.join(
        results_dir,
        "weights",
        chiron.utils.get_weights_path().format(epoch=epoch),
    )
)
config = chiron.utils.load_yaml(os.path.join(results_dir, "config.yaml"))
image_size = tf.TensorShape(config["image_size"])
model.build([None] + image_size + [3])
model.summary()


In [ ]:
dataset = (
    chiron.tfrecord.load_tfrecord(tfrecord_file)
    .map(chiron.preprocessing.Resizer(image_size))
    .batch(batch_size)
    .map(chiron.preprocessing.PerImageStandardizer())
    .map(chiron.preprocessing.GrayscaleToRgb())
)


def generate():
    for images, _ in dataset:
        with tf.GradientTape() as tape:
            tape.watch(images)
            scores = tf.nn.softmax(model(images, training=False))
            losses = tf.reduce_max(scores, axis=1)
        gradients = tape.gradient(losses, images)
        s_maps = tf.reduce_max(gradients, axis=3)
        yield from zip(images.numpy(), s_maps.numpy())


gen = generate()


In [ ]:
image, s_map = next(gen)

image = image - image.min()
image *= 255 / image.max()
image = image.astype(np.uint8)
image = Image.fromarray(image)

s_map = gaussian_filter(s_map, 5)
threshold = np.percentile(s_map, 95)
s_map = np.ma.masked_less(s_map, threshold, copy=False)
s_map -= s_map.min()
s_map *= 1 / s_map.max()
s_map = get_cmap("jet")(s_map, alpha=0.5)
s_map *= 255
s_map = s_map.astype(np.uint8)
s_map = Image.fromarray(s_map)
s_map

Image.alpha_composite(image.convert('RGBA'), s_map)
